## 2. Data Preparation
📒 `2.0-rc-data-preparation.ipynb`

**Objetivo:** Transformar os dados brutos em um formato para uso em análise exploratória e modelagem.

⚙️ **Atividades:**
- Limpeza dos dados
- Feature engineering
- Encoding
- Escalonamento
- Split do dataset.
- Salvamento do dataset limpo (`data/processed/`)


## 📘 Dicionário de Dados  

| CAMPO                  | FUNÇÃO (ATRIBUTO \| ALVO) | TIPO DE VARIÁVEL           | DESCRIÇÃO |
|-------------------------|---------------------------|----------------------------|-----------|
| **id**                 | ID                 | Numérica (Inteiro)         | Identificador único das amostras do banco de dados. Não possui significado físico, usado apenas para indexação. |
| **id_produto**         | ID                  | Categórica (String)        | Identificador do produto/máquina, combinação da variável `tipo` com um número sequencial. Representa diferentes unidades operacionais. |
| **tipo**               | Atributo                  | Categórica (L, M, H)       | Tipo de produto/máquina, definido por categoria de carga: **L** (Low), **M** (Medium), **H** (High). No dataset AI4I, simula variação entre diferentes condições de produção. |
| **temperatura_ar**     | Atributo                  | Numérica (Contínua, K)     | Temperatura ambiente medida em Kelvin. Importante para avaliar condições externas que influenciam o desempenho térmico da máquina. |
| **temperatura_processo** | Atributo                | Numérica (Contínua, K)     | Temperatura do processo (interno à máquina). Valores mais altos estão associados a falhas de dissipação de calor (FDC). |
| **umidade_relativa**   | Atributo                  | Numérica (Contínua, %)     | Umidade relativa do ar (%). Pode impactar falhas de isolamento elétrico e desgaste acelerado. |
| **velocidade_rotacional** | Atributo               | Numérica (Contínua, RPM)   | Velocidade rotacional da máquina em rotações por minuto (RPM). Valores anormais podem indicar problemas de balanceamento ou sobrecarga. |
| **torque**             | Atributo                  | Numérica (Contínua, Nm)    | Torque da máquina em Newton-metro. No dataset AI4I varia entre ~3.0 a 76.0 Nm. Torques elevados podem levar a falhas de potência (FP). |
| **desgaste_da_ferramenta** | Atributo              | Numérica (Contínua, Min)   | Tempo acumulado de uso da ferramenta (minutos). Aumenta linearmente com a operação e está associado a falhas por desgaste (FDF). |
| **falha_maquina**      | Alvo (Binário)            | Numérica (0/1)             | Variável binária indicando se houve falha geral na máquina (1) ou não (0). Serve como rótulo consolidado de falha. |
| **FDF** (Falha Desgaste Ferramenta) | Alvo (Binário) | Numérica (0/1)            | Indica falha por desgaste excessivo da ferramenta de corte. Associada ao tempo de uso (`desgaste_da_ferramenta`) e às condições de torque/RPM. |
| **FDC** (Falha Dissipação Calor)    | Alvo (Binário) | Numérica (0/1)            | Indica falha térmica por dissipação inadequada de calor. Relacionada principalmente a `temperatura_processo` elevada em relação à `temperatura_ar`. |
| **FP** (Falha Potência)             | Alvo (Binário) | Numérica (0/1)            | Indica falha por potência insuficiente ou sobrecarga da máquina. Frequentemente correlacionada com variações de torque e rotação. |
| **FTE** (Falha Tensão Excessiva)    | Alvo (Binário) | Numérica (0/1)            | Indica falha elétrica devido a sobretensão ou condições anômalas de operação elétrica. |
| **FA** (Falha Aleatória)            | Alvo (Binário) | Numérica (0/1)            | Indica falha aleatória sem padrão determinístico. No dataset, essas falhas são geradas para simular eventos não explicados por variáveis medidas. |


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline

# Setup para mostrar todas as colunas do dataframe
pd.set_option('display.max_columns', None)

In [5]:
# Carrega dados
df = pd.read_csv("../data/raw/bootcamp_train.csv")
df.head()

,id,id_produto,tipo,temperatura_ar,temperatura_processo,umidade_relativa,velocidade_rotacional,torque,desgaste_da_ferramenta,falha_maquina,FDF (Falha Desgaste Ferramenta),FDC (Falha Dissipacao Calor),FP (Falha Potencia),FTE (Falha Tensao Excessiva),FA (Falha Aleatoria)
0,0,L56434,L,298.3,309.1,90.0,1616.0,31.1,195.0,não,False,False,Não,False,Não
1,1,L48741,L,298.2,308.4,90.0,1388.0,53.8,137.0,Não,False,False,Não,False,Não
2,2,L48850,L,298.2,307.8,90.0,1528.0,31.1,NaN,Não,N,False,Não,False,Não
3,3,M20947,M,300.9,310.8,90.0,1599.0,33.0,7.0,não,False,False,Não,False,não
4,4,L53849,L,-36.0,310.5,90.0,1571.0,33.9,NaN,não,N,False,não,False,Não


### 1. Limpeza dos Dados

#### a) Valores Nulos

In [ ]:
# Verifica dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35260 entries, 0 to 35259
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               35260 non-null  int64  
 1   id_produto                       35260 non-null  object 
 2   tipo                             35260 non-null  object 
 3   temperatura_ar                   34644 non-null  float64
 4   temperatura_processo             34661 non-null  float64
 5   umidade_relativa                 35260 non-null  float64
 6   velocidade_rotacional            34509 non-null  float64
 7   torque                           34637 non-null  float64
 8   desgaste_da_ferramenta           34308 non-null  float64
 9   falha_maquina                    35260 non-null  object 
 10  FDF (Falha Desgaste Ferramenta)  35260 non-null  object 
 11  FDC (Falha Dissipacao Calor)     35260 non-null  object 
 12  FP (Falha Potencia

In [7]:
# Verifica dados Nulos
df.isnull().sum()

id                                   0
id_produto                           0
tipo                                 0
temperatura_ar                     616
temperatura_processo               599
umidade_relativa                     0
velocidade_rotacional              751
torque                             623
desgaste_da_ferramenta             952
falha_maquina                        0
FDF (Falha Desgaste Ferramenta)      0
FDC (Falha Dissipacao Calor)         0
FP (Falha Potencia)                  0
FTE (Falha Tensao Excessiva)         0
FA (Falha Aleatoria)                 0
dtype: int64

In [ ]:
# Verifica dados Nulos
df.isnull().sum()

# Verifica dados Duplicados
df.duplicated().sum()

# Verifica dados Inconsistentes
df.describe()

# Verifica dados Outliers
df.boxplot()

In [ ]:
# Salva o dataset tratado
path_to_save = "../data/processed/bootcamp_train_processed.csv"
df.to_csv(path_to_save, index=False)